### Install dependencies

In [ ]:
    !pip install beautifulsoup4==4.13.3, datasets==3.3.1 langchain-cohere==0.4.2 langchain-community==0.3.14 langchain-huggingface==0.1.2 -q
    !pip install langchain-openai==0.2.14 langchain-qdrant==0.2.0 langgraph==0.2.61  pymupdf==1.25.3 ragas==0.2.10 sentence-transformers==3.4.1 pypdf -q


In [ ]:
pip install --upgrade langchain


### Openai API and Tavily API

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")


Enter Your OpenAI API Key: ··········


In [ ]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter Your Tavily API Key: ")

Enter Your Tavily API Key: ··········


In [ ]:
from typing import TypedDict, Annotated, List
from typing_extensions import List, TypedDict

from dotenv import load_dotenv
import operator

from langchain.prompts import ChatPromptTemplate
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langgraph.graph import START, StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams



### Load and chunk data


In [ ]:
load_dotenv()

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

path = "data/"
text_loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyPDFLoader)

In [ ]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 200,
    length_function = len
)

In [ ]:
import re
def remove_references(doc):
    text = doc.page_content

    # Common headers for reference sections
    reference_markers = ["References", "Bibliography", "Cited Works", "Literature Cited"]

    for marker in reference_markers:
        if marker in text:
            text = text.split(marker)[0]  # Keep only the content before references
            break  # Stop checking after the first match

    # 2️⃣ Eliminar DOI, enlaces y citas tipo [1], [2], etc.
    text = re.sub(r"https?://\S+|doi:\S+", "", text)
    text = re.sub(r"\[\d+\]", "", text)  # Remueve referencias numéricas en corchetes

    # 3️⃣ Eliminar saltos de línea innecesarios
    text = re.sub(r"\n{2,}", "\n", text).strip()
    doc.page_content = text.strip()  # Update document content
    return doc

# Apply reference filtering
filtered_documents = [remove_references(doc) for doc in text_loader.load()]

training_documents = text_splitter.split_documents(filtered_documents)

### Set up vector database

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="Gonalb/flucold-ft-v2")


client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=training_documents)

retriever = vector_store.as_retriever(search_kwargs={"k": 6})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at Gonalb/flucold-ft-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### RAG

In [ ]:
# ----------------- State -----------------
class AgentState(TypedDict):
    messages: Annotated[list, "add_messages"]
    question: str
    context: List[Document]

# ----------------- RAG Components -----------------
def retrieve(state):
    """Recupera documentos relevantes con el retriever."""
    retrieved_docs = retriever.invoke(state["question"])
    return {"context": retrieved_docs}

RAG_PROMPT = """\
You are a helpful AI-powered Flu & Respiratory Illness Consultant. Your job is to help users determine whether they have the flu, a cold, RSV, or allergies based on their symptoms.
Provide recommendations based on the context provided. If symptoms are severe, advise the user to seek medical attention.
Avoid giving definitive diagnoses or prescriptions—always encourage users to consult a healthcare professional for serious cases.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)
llm = ChatOpenAI(model="gpt-4o")

def generate(state):
    """Genera una respuesta basada en los documentos recuperados."""
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
    response = llm.invoke(messages)
    return {"messages": [response]}


### Tool and graph

In [ ]:

# ----------------- Tools & Agent -----------------
tavily_tool = TavilySearchResults(max_results=5)
tool_belt = [tavily_tool]
model = ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(tool_belt)
tool_node = ToolNode(tool_belt)

def call_model(state):
    """Llama al modelo base para generar respuestas."""
    messages = state["messages"]
    response = model.invoke(messages)
    return {
        "messages": [response],
        "question": state["question"],
        "context": state.get("context", [])
    }

# ----------------- Graph -----------------
uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("retrieve", retrieve)
uncompiled_graph.add_node("generate", generate)
uncompiled_graph.add_node("action", tool_node)

uncompiled_graph.set_entry_point("retrieve")

# ----------------- Control logic -----------------
def should_continue(state):
    """Decide si usar herramientas después de `generate`."""
    last_message = state["messages"][-1]

    if last_message.tool_calls:
        return "action"

    return END

uncompiled_graph.add_edge("retrieve", "generate")
uncompiled_graph.add_conditional_edges("generate", should_continue)
uncompiled_graph.add_edge("action", "generate")

compiled_graph = uncompiled_graph.compile()


In [ ]:
compiled_graph.invoke({
        "messages": [HumanMessage(content='How long does the flu last?')],
         "question": 'How long does the flu last?',
         "context": []
     })

{'messages': [AIMessage(content="The flu typically lasts about one to two weeks. The acute symptoms, such as fever, headache, and muscle aches, often appear quickly and can be most severe during the first three to four days. As these symptoms start to resolve, other symptoms like cough and fatigue may persist for a longer period, sometimes extending beyond a week. \n\nIf you have severe symptoms or are at high risk of complications, it's important to seek medical attention. Antiviral medications can be prescribed to reduce the duration of the flu if taken within 48 hours of symptom onset, and are most effective when started within 24 hours. Always consider consulting a healthcare professional for a proper assessment and treatment plan.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 137, 'prompt_tokens': 875, 'total_tokens': 1012, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction